In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

## Training Data

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
x_train.shape, y_train.shape

(torch.Size([6, 2]), torch.Size([6, 1]))

## computing the Hypothesis
$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
W = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [9]:
hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)

In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b); hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)

## 1element cost

In [22]:
-(y_train[0] * torch.log(hypothesis[0]) +
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [24]:
## all hypothesis
losses = -(y_train * torch.log(hypothesis) +
  (1 - y_train) * torch.log(1 - hypothesis)); losses

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)

In [26]:
cost = losses.mean(); cost

tensor(0.6931, grad_fn=<MeanBackward0>)

## Cost Funtion

In [27]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

## Training

In [134]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [135]:
W = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch {:4d} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

epoch    0 / 1000 Cost: 0.693147
epoch  100 / 1000 Cost: 0.134722
epoch  200 / 1000 Cost: 0.080643
epoch  300 / 1000 Cost: 0.057900
epoch  400 / 1000 Cost: 0.045300
epoch  500 / 1000 Cost: 0.037261
epoch  600 / 1000 Cost: 0.031673
epoch  700 / 1000 Cost: 0.027556
epoch  800 / 1000 Cost: 0.024394
epoch  900 / 1000 Cost: 0.021888
epoch 1000 / 1000 Cost: 0.019852


## Evaluation

In [136]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
hypothesis

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward>)

In [137]:
predict = hypothesis >= 0.5; predict

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

In [138]:
a = predict == y_train; a

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])

In [144]:
a.sum()

tensor(6)

In [145]:
a.sum(0).item()

6

In [285]:
## Loading Real Data
import numpy as np
xy = np.loadtxt('data-03-diabetes.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]] # 2차원으로 만들어주긔
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [97]:
print(x_train[:5])
print(y_train[:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [119]:
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693148
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488208
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


In [123]:
W = torch.zeros((x_train.shape[1], 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W, b], lr = 1)
nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
#     hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/100 Cost: 0.693148
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488208
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


In [130]:
W = torch.zeros((x_train.shape[1], 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W, b], lr = 1)
nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
#     hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539494
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507065
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488208
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


In [410]:
c = nn.Linear(2, 1, bias = False)

In [429]:
e = nn.Linear(2, 1)

In [428]:
list(c.parameters())

[Parameter containing:
 tensor([[ 0.3663, -0.0691]], requires_grad=True)]

In [430]:
list(e.parameters())

[Parameter containing:
 tensor([[-0.2780,  0.3373]], requires_grad=True), Parameter containing:
 tensor([-0.4698], requires_grad=True)]

In [422]:
print(c(torch.FloatTensor([[2, 1],
                           [1, 2]])))
print(c(torch.FloatTensor([2, 2])))
print(c(torch.FloatTensor([1, 2])))
print(c(torch.FloatTensor([2, 1])))
print(c(torch.FloatTensor([1, 1])))
print(c(torch.FloatTensor([0, 1])))
print(c(torch.FloatTensor([1, 0])))
print(c(torch.FloatTensor([0, 0])))

tensor([[0.6634],
        [0.2281]], grad_fn=<MmBackward>)
tensor([0.5943], grad_fn=<SqueezeBackward3>)
tensor([0.2281], grad_fn=<SqueezeBackward3>)
tensor([0.6634], grad_fn=<SqueezeBackward3>)
tensor([0.2972], grad_fn=<SqueezeBackward3>)
tensor([-0.0691], grad_fn=<SqueezeBackward3>)
tensor([0.3663], grad_fn=<SqueezeBackward3>)
tensor([0.], grad_fn=<SqueezeBackward3>)


In [417]:
a, b = 0.36, -0.06
(2 * a) + (2 * b)

0.6

## High level

In [437]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward1(self, x):
        return self.sigmoid(self.linear(x))

In [438]:
model = BinaryClassifier() # 이렇게 안 해두면 W를 랜덤으로 뽑는데
# BinaryClassifier가 쓰이는 곳마다 값이 다르게 뽑힘

In [355]:
a = 1 or 2

[Parameter containing:
 tensor([[ 0.2965,  0.1312,  0.3353, -0.0538,  0.2726,  0.3244,  0.2971, -0.1981]],
        requires_grad=True), Parameter containing:
 tensor([0.3513], requires_grad=True)]

In [ ]:
q = a

In [376]:
list(model.parameters())[1]

Parameter containing:
tensor([0.2731], requires_grad=True)

In [373]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2577, -0.0530,  0.0415,  0.1458,  0.2747, -0.2860,  0.3364,  0.0461]],
        requires_grad=True), Parameter containing:
 tensor([0.2807], requires_grad=True)]

In [377]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
#     hypothesis = model(x_train)
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))
        print(cost)

Epoch    0/100 Cost: 0.479583 Accuracy 77.21%
tensor(0.4796, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   10/100 Cost: 0.478244 Accuracy 77.08%
tensor(0.4782, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   20/100 Cost: 0.477191 Accuracy 76.81%
tensor(0.4772, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   30/100 Cost: 0.476352 Accuracy 76.81%
tensor(0.4764, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   40/100 Cost: 0.475676 Accuracy 76.81%
tensor(0.4757, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   50/100 Cost: 0.475126 Accuracy 77.08%
tensor(0.4751, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   60/100 Cost: 0.474675 Accuracy 77.08%
tensor(0.4747, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   70/100 Cost: 0.474301 Accuracy 77.08%
tensor(0.4743, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   80/100 Cost: 0.473989 Accuracy 77.08%
tensor(0.4740, grad_fn=<BinaryCrossEntropyBackward>)
Epoch   90/100 Cost: 0.473724 Accuracy 77.08%
tensor(0.4737, grad_fn=<BinaryCrossEntropyBackward>)
Epoch  100